In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [2]:
# Własne importy:
from content_collection import collect_serp_data_and_extract_text_from_webpages
from custom_summarize_chain import create_all_summaries, DocumentSummary
from expert_interview_chain import InterviewChain

## Badanie tematu:

In [3]:
%pip install google-search-results pandas html2text pytest-playwright chromadb nest_asyncio --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
!playwright install

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
# Stałe zmienne:
TOPIC = "Sieci neuronowe"
os.environ["SERPAPI_API_KEY"] = "2"
os.environ["STABILITY_API_KEY"] = "sk-gxVY1Jl2ZCspR5hOXp7rsqTlEUlvG2gSrrTepwnAbqOYmEy5"

In [7]:
# Przetwórz treść stron internetowych do postaci dokumentów LangChain:
text_documents = await collect_serp_data_and_extract_text_from_webpages(topic=TOPIC)

In [8]:
# LLM, rozdzielacz tekstu + parser:
llm = ChatOpenAI(temperature=0)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=400
)
parser = PydanticOutputParser(pydantic_object=DocumentSummary)

In [9]:
summaries = await create_all_summaries(text_documents, parser, llm, text_splitter)

first
page_content='Przejdź do zawartości\n\nMenu główne\n\nMenu główne\n\nprzypnij ukryj\n\nNawigacja\n\n  * Strona główna\n  * Losuj artykuł\n  * Kategorie artykułów\n  * Najlepsze artykuły\n  * Częste pytania (FAQ)\n\nDla czytelników\n\n  * O Wikipedii\n  * Kontakt\n\nDla wikipedystów\n\n  * Pierwsze kroki\n  * Portal wikipedystów\n  * Ogłoszenia\n  * Zasady\n  * Pomoc\n  * Ostatnie zmiany\n\nSzukaj\n\nSzukaj\n\n  * Wspomóż Wikipedię\n\nWygląd\n\n  * Utwórz konto\n  * Zaloguj się\n\nNarzędzia osobiste\n\n  * Utwórz konto\n  * Zaloguj się\n\nStrony dla anonimowych edytorów dowiedz się więcej\n\n  * Edycje\n  * Dyskusja\n\n## Spis treści\n\nprzypnij ukryj\n\n  * Początek\n\n  * 1 Typy sieci neuronowych\n\nPrzełącz podsekcję Typy sieci neuronowych\n\n    * 1.1 Sieci jednokierunkowe\n\n    * 1.2 Sieci rekurencyjne\n\n    * 1.3 Samoorganizujące się mapy\n\n    * 1.4 Inne\n\n  * 2 Wybrane narzędzia do budowy sieci neuronowych\n\nPrzełącz podsekcję Wybrane narzędzia do budowy sieci neurono

---

## Expert Interview Questions:


In [10]:
interview_chain = InterviewChain(topic=TOPIC, document_summaries=summaries)
interview_questions = interview_chain()

----------------------------------------

## Answer The Interview Questions:

In [11]:
for question in interview_questions.questions:
    print(f"Odpowiedz na następujące pytanie: {question.question}\n", flush=True)
    answer = input(f"Odpowiedz na następujące pytanie: {question.question}\n")
    print('------------------------------------------')
    question.answer = answer

Odpowiedz na następujące pytanie: Jakie są główne typy sieci neuronowych i w jaki sposób się od siebie różnią?



Odpowiedz na następujące pytanie: Jakie są główne typy sieci neuronowych i w jaki sposób się od siebie różnią?
 Są trzy typy, nie różnią się


------------------------------------------
Odpowiedz na następujące pytanie: Jakie narzędzia są najczęściej wykorzystywane do budowy sieci neuronowych i dlaczego?



Odpowiedz na następujące pytanie: Jakie narzędzia są najczęściej wykorzystywane do budowy sieci neuronowych i dlaczego?
 Tektura i gips


------------------------------------------
Odpowiedz na następujące pytanie: W jaki sposób sieć neuronowa zdobywa wiedzę i jak przechowuje ją?



...

------------------------------------------
Odpowiedz na następujące pytanie: Czy istnieją trudności z interpretacją wag synaptycznych w sieciach neuronowych? Jeśli tak, to dlaczego?



Odpowiedz na następujące pytanie: Czy istnieją trudności z interpretacją wag synaptycznych w sieciach neuronowych? Jeśli tak, to dlaczego?
 Nie ma trudności


------------------------------------------
Odpowiedz na następujące pytanie: W jaki sposób sieci neuronowe mogą być wykorzystane w praktyce w różnych dziedzinach?



Odpowiedz na następujące pytanie: W jaki sposób sieci neuronowe mogą być wykorzystane w praktyce w różnych dziedzinach?
 Nie mogą być


------------------------------------------


---

## General Article Outline:


In [12]:
from article_outline_generation import BlogOutlineGenerator

blog_outline_generator = BlogOutlineGenerator(topic=TOPIC, questions_and_answers=[item.dict()  
                                                                                  for item in interview_questions.questions ] )
questions_and_answers = blog_outline_generator.questions_and_answers
outline_result = blog_outline_generator.generate_outline(summaries)

Generowanie zarysu...
---
Zakończono generowanie zarysu!
---


---

## Article Text Generation:


In [13]:
from article_generation import ContentGenerator

content_gen = ContentGenerator(topic=TOPIC, outline=outline_result, questions_and_answers=questions_and_answers)

# Wektoryzuj i przechowuj źródłowe strony internetowe:
content_gen.split_and_vectorize_documents(text_documents)

# # Utwórz post na blogu (może zająć kilka minut – prośba o cierpliwość):
blog_post = content_gen.generate_blog_post()

Generowanie posta na bloga...
---
Zakończono generowanie posta na bloga!
---


In [14]:
blog_post

['# Sieć neuronowa jako model matematyczny inspirowany działaniem mózgu\n\nSieć neuronowa to masowo równoległy procesor rozproszony, który jest naturalnie predystynowany do gromadzenia wiedzy eksperymentalnej i udostępniania jej w celu wykorzystania. Sieć neuronowa pod dwoma względami przypomina mózg:\n- Sieć zdobywa wiedzę, ucząc się.\n- Do przechowywania wiedzy wykorzystywane są siły połączeń między neuronami, nazywane wagami synaptycznymi.\n\nPierwsza sieć neuronowa została opracowana przez Warrena McCullocha i Waltera Pittsa w 1943 roku. Napisali oni przełomowy artykuł na temat tego, jak mogą działać neurony, i zademonstrowali swoje pomysły, tworząc prostą sieć neuronową za pomocą obwodów elektrycznych. Ten przełomowy model utorował drogę badaniom nad sieciami neuronowymi w dwóch obszarach: procesy biologiczne w mózgu oraz zastosowanie sieci neuronowych powiązane ze sztuczną inteligencją (AI).\n\nSieć neuronowa jest systemem przeznaczonym do przetwarzania informacji, którego budowa

-----

## Image Creation:

In [15]:
from image_generation_chain import create_image
image = create_image(outline_result.title)

Enter your Stability API key:  ········


In [ ]:
print(f"Obrazek jest gotowy! Ścieżka do obrazku to {image[0]}")